# 11.2. Attention Pooling by Similarity

## 11.2.1. Kernels and Data

Attention Pooling by Similarity 是一种基于相似度的注意力汇聚方法，它将传统的注意力机制与特征汇聚（pooling）结合在一起，通过计算相似度来为输入序列中的每个元素分配不同的权重，从而实现更加灵活和自适应的汇聚方式。

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

d2l.use_svg_display()

在 D2L（Dive into Deep Learning）课程的第 11.2 章中，提到了注意力机制中的“内核”（Kernel）和“数据”（Data）两个概念，这里的“内核”指的是不同的核函数（kernel functions）用于计算相似性或注意力权重。以下是对几种内核的解释：

### 1. **高斯内核（Gaussian Kernel）**
高斯内核（也称为径向基函数核，RBF kernel）是最常见的一种核函数，它根据输入向量的距离来计算相似性。对于注意力机制，**高斯内核**的作用是根据查询向量和键向量之间的距离来计算权重，较近的输入项会分配较高的权重，远的项权重较低。

公式如下：
$$
K(x, x') = \exp \left( - \frac{{\| x - x' \|^2}}{{2 \sigma^2}} \right)
$$
其中，\( x \) 和 \( x' \) 分别代表查询和键向量，\( \sigma \) 控制了内核的宽度。

**特点**：
- 对距离敏感，能够捕捉局部相似性。
- 通常用于平滑处理，可以生成平滑的注意力权重分布。

### 2. **Boxcar 内核**
Boxcar 内核是一种简单的、非平滑的内核。它只对在一定范围内的输入分配相等的权重，范围之外的输入直接忽略。它的形式类似于一个矩形窗口，因此也称为矩形内核或窗口内核。

公式如下：
$$
K(x, x') = 
\begin{cases}
1, & \text{if } |x - x'| \leq h \\
0, & \text{otherwise}
\end{cases}
$$
其中 \( h \) 是窗口的宽度参数。

**特点**：
- 在定义的范围内，所有元素得到相等的权重，没有平滑效果。
- 超出范围的元素直接忽略。

### 3. **常数内核（Constant Kernel）**
常数内核是最简单的一种内核函数，它为所有输入分配相等的注意力权重，完全忽略查询和键向量之间的相似性或距离。因此，它在所有位置上都分配了相同的权重，没有任何偏向。

公式如下：
$$
K(x, x') = c
$$
其中 \( c \) 是一个常数，可以设定为 1。

**特点**：
- 没有相似性或距离信息的考虑，分配均匀的权重。
- 非常简单，但是忽略了序列中的差异性，通常不适合复杂的场景。

### 4. **Epanechikov 内核**
Epanechikov 内核是一种常见的核函数，用于平滑数据。它也被称为抛物线核或二次核。与高斯核类似，它也随着输入项之间的距离增加而减小权重，但是它在距离为 0 的地方分配最大的权重，然后随着距离的增加呈二次方下降。

公式如下：
$$
K(x, x') = 
\begin{cases}
\frac{3}{4}(1 - \frac{{(x - x')^2}}{h^2}), & \text{if } |x - x'| \leq h \\
0, & \text{otherwise}
\end{cases}
$$
其中 \( h \) 是带宽参数。

**特点**：
- 对近邻数据分配较高权重，并且权重呈抛物线形式逐渐衰减。
- 比高斯核更为局部化，适合捕捉局部特征。

### **总结内核的作用**

这些内核函数在注意力机制中的作用是决定如何根据查询和键向量之间的相似性或距离来分配注意力权重。不同的核函数可以产生不同的权重分布，从而影响模型对不同输入元素的关注程度：

- **高斯内核**更适合平滑的注意力分布。
- **Boxcar 内核**用于分配固定范围内的均匀权重。
- **常数内核**忽略距离信息，分配相同的权重。
- **Epanechikov 内核**用于平滑且局部化的权重分布。

这些核函数提供了不同的机制来处理序列数据中的相似性关系，适用于不同的任务和场景。


In [ ]:
# Define some kernels
def gaussian(x):
    return torch.exp(-x**2 / 2)

def boxcar(x):
    return torch.abs(x) < 1.0

def constant(x):
    return 1.0 + 0 * x

def epanechikov(x):
    return torch.max(1 - torch.abs(x), torch.zeros_like(x))

fig, axes = d2l.plt.subplots(1, 4, sharey=True, figsize=(12, 3))

kernels = (gaussian, boxcar, constant, epanechikov)
names = ('Gaussian', 'Boxcar', 'Constant', 'Epanechikov')
x = torch.arange(-2.5, 2.5, 0.1)
for kernel, name, ax in zip(kernels, names, axes):
    ax.plot(x.detach().numpy(), kernel(x).detach().numpy())
    ax.set_xlabel(name)

d2l.plt.show()

In [ ]:
def f(x):
    return 2 * torch.sin(x) + x

In [ ]:
n = 40
x_train, _ = torch.sort(torch.rand(n) * 5)
x_train

In [ ]:

y_train = f(x_train) + torch.randn(n)
y_train


In [ ]:
x_val = torch.arange(0, 5, 0.1)
x_val


In [ ]:
y_val = f(x_val)
y_val